In [2]:
cd ../..

/home/nazif/thesis/mirscribe-vcf


/home/nazif/.cache/pypoetry/virtualenvs/mirscribe-vcf-8VJm4tPg-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import sqlite3
import pandas as pd

from scripts.pipeline_steps import step5 as s5
from scripts.pyensembl_operations import import_pyensembl
g37 = import_pyensembl(37)

from scripts.notebook_operations import setup_notebook

engine, g37, transcripts, genes, mirnas = setup_notebook()


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


# functions

In [2]:
def drop_table(db_path, table_name):
    """
    Drop a table from a SQLite database.

    Args:
        db_path (str): The path to the SQLite database file.
        table_name (str): The name of the table to drop.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    # Drop the table
    c.execute(f"DROP TABLE IF EXISTS {table_name}")

    conn.commit()
    conn.close()

# import real data

In [3]:
drop_table('data/db/mirscribe.db', 'predictions')

In [ ]:
s5.crawl_and_import_results('results/21/', 'a', 'data/db/mirscribe.db', 'predictions', g37)

# import synthetic data

In [ ]:
drop_table('data/db/mirscribe.db', 'synth')


In [ ]:
s5.crawl_and_import_results(folder_path='results/synth/', ending_string='a+', db_path='data/db/mirscribe.db', table_name='synth', assembly=g37)


# post processing

In [5]:
df = pd.read_sql_table("predictions", engine)

df = df.merge(genes, on="gene_id", how="left")
df = df.merge(mirnas, on="mirna_accession", how="left")
df.drop(columns=["chr", "start", "end", "sequence", "conservation", "mirna_name_cmc"], inplace=True)

# fill NAs
df["mutsig"] = df["mutsig"].fillna("not_found")
df["gene_id"] = df["gene_id"].fillna("not_found")
df["gene_name"] = df["gene_name"].fillna("not_found")
df["gene_biotype"] = df["gene_biotype"].fillna("not_found")
df["gene_description"] = df["gene_description"].fillna("not_found")
df["tier_cosmic"] = df["tier_cosmic"].fillna(0)
df = df.fillna(False)

df.to_parquet("data/real.parquet")

/tmp/ipykernel_41470/1280123063.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


In [6]:
fg = pd.read_sql_table("synth", engine)

fg = fg.merge(genes, on="gene_id", how="left")
fg = fg.merge(mirnas, on="mirna_accession", how="left")
fg.drop(columns=["chr", "start", "end", "sequence", "conservation", "mirna_name_cmc"], inplace=True)

# fill NAs
fg["mutsig"] = fg["mutsig"].fillna("not_found")
fg["gene_id"] = fg["gene_id"].fillna("not_found")
fg["gene_name"] = fg["gene_name"].fillna("not_found")
fg["gene_biotype"] = fg["gene_biotype"].fillna("not_found")
fg["gene_description"] = fg["gene_description"].fillna("not_found")
fg["tier_cosmic"] = fg["tier_cosmic"].fillna(0)
fg = fg.fillna(False)

fg.to_parquet("data/synth.parquet")

/tmp/ipykernel_41470/2549095794.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fg = fg.fillna(False)
